# 176: Fairness Bias in ML

In [ ]:
"""
Fairness & Bias in ML Environment Setup
========================================

Purpose: Import libraries for fairness analysis and bias mitigation.

Key Libraries:
- fairlearn: Microsoft's fairness toolkit
- aif360: IBM's AI Fairness 360 toolkit
- scikit-learn: ML models and metrics
- numpy/pandas: Data manipulation

Why This Matters:
- Production-ready fairness tools
- Industry-standard metrics and mitigation algorithms
- Regulatory compliance support
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, classification_report
)

# Fairness Libraries (optional - will implement from scratch)
try:
    from fairlearn.metrics import (
        demographic_parity_difference,
        equalized_odds_difference,
        MetricFrame
    )
    from fairlearn.reductions import ExponentiatedGradient, DemographicParity
    FAIRLEARN_AVAILABLE = True
except ImportError:
    print("⚠️ fairlearn not installed. Using custom implementations.")
    print("   Install: pip install fairlearn")
    FAIRLEARN_AVAILABLE = False

# Visualization
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Random seed
np.random.seed(42)

print("✅ Fairness & Bias Environment Ready!")
print("\nKey Capabilities:")
print("  - Bias detection (demographic parity, equal opportunity)")
print("  - Fairness metrics (20+ metrics implemented)")
print("  - Debiasing techniques (pre/in/post-processing)")
print("  - Fairness-accuracy tradeoff analysis")
print("  - Regulatory compliance reporting")
print(f"\nFairlearn library: {'✅ Available' if FAIRLEARN_AVAILABLE else '❌ Not installed (using custom)'}")

## 🧮 Mathematical Foundation: Fairness Metrics

### **1. Demographic Parity (Statistical Parity)**

**Definition:** Positive prediction rates are equal across protected groups.

$$
P(\hat{Y} = 1 | A = 0) = P(\hat{Y} = 1 | A = 1)
$$

Where:
- $\hat{Y}$: Model prediction (0 = negative, 1 = positive)
- $A$: Protected attribute (0 = privileged group, 1 = unprivileged group)

**Metric:** Demographic Parity Difference (DPD)

$$
\text{DPD} = P(\hat{Y} = 1 | A = 1) - P(\hat{Y} = 1 | A = 0)
$$

**Interpretation:**
- $\text{DPD} = 0$: Perfect demographic parity
- $|\text{DPD}| < 0.1$: Acceptable fairness (industry guideline)
- $|\text{DPD}| > 0.2$: Significant bias

**When to Use:**
- Loan approvals, hiring decisions, resource allocation
- When **equal selection rates** are desired regardless of ground truth

**Limitation:**
- Ignores prediction accuracy
- May conflict with Equal Opportunity

---

### **2. Equal Opportunity**

**Definition:** True positive rates (TPR) are equal across protected groups.

$$
P(\hat{Y} = 1 | Y = 1, A = 0) = P(\hat{Y} = 1 | Y = 1, A = 1)
$$

Where:
- $Y$: Ground truth label (0 = negative, 1 = positive)

**Metric:** Equal Opportunity Difference (EOD)

$$
\text{EOD} = \text{TPR}_{A=1} - \text{TPR}_{A=0}
$$

$$
\text{TPR}_A = \frac{\text{TP}_A}{\text{TP}_A + \text{FN}_A}
$$

**Interpretation:**
- $\text{EOD} = 0$: Perfect equal opportunity
- $|\text{EOD}| < 0.1$: Acceptable
- $|\text{EOD}| > 0.15$: Concerning bias

**When to Use:**
- Medical diagnosis, fraud detection, disease screening
- When **correctly identifying positive cases** is critical for all groups

**Advantage:**
- Accounts for ground truth (more meaningful than Demographic Parity)

---

### **3. Equalized Odds**

**Definition:** Both TPR and FPR are equal across protected groups.

$$
P(\hat{Y} = 1 | Y = y, A = 0) = P(\hat{Y} = 1 | Y = y, A = 1) \\quad \\forall y \\in \\{0, 1\\}
$$

**Metric:** Equalized Odds Difference (max of TPR and FPR differences)

$$
\text{Equalized Odds Diff} = \\max(|\text{TPR}_{A=1} - \text{TPR}_{A=0}|, |\text{FPR}_{A=1} - \text{FPR}_{A=0}|)
$$

**When to Use:**
- Criminal justice, credit scoring
- When **both false positives and false negatives** have serious consequences

---

### **4. Calibration (Predictive Parity)**

**Definition:** Predicted probabilities match true positive rates across groups.

$$
P(Y = 1 | \\hat{Y} = s, A = 0) = P(Y = 1 | \\hat{Y} = s, A = 1) \\quad \\forall s
$$

**Metric:** Calibration Difference (across probability bins)

$$
\text{Calibration Diff} = \\max_s |P(Y=1|\\hat{Y}=s, A=1) - P(Y=1|\\hat{Y}=s, A=0)|
$$

**Example:**
- If model predicts 70% default probability for both groups
- **Calibrated:** 70% of each group actually defaults
- **Not calibrated:** 70% of Group A defaults, 50% of Group B defaults

**When to Use:**
- Risk assessment where probability interpretation matters
- Insurance, credit scoring

---

### **5. Individual Fairness**

**Definition:** Similar individuals receive similar predictions.

$$
d(\\hat{y}_i, \\hat{y}_j) \\leq L \\cdot d(x_i, x_j)
$$

Where:
- $d(\\hat{y}_i, \\hat{y}_j)$: Distance between predictions
- $d(x_i, x_j)$: Distance between feature vectors
- $L$: Lipschitz constant (controls sensitivity)

**Challenge:** Defining \"similarity\" is subjective and domain-dependent

---

### **6. Impossibility Theorem**

**Chouldechova (2017) & Kleinberg et al. (2017):**

**Cannot simultaneously satisfy:**
1. **Calibration** (predictive parity)
2. **Equal Opportunity** (equal TPR)
3. **Demographic Parity** (equal selection rate)

**Unless:** Base rates are equal across groups: $P(Y=1|A=0) = P(Y=1|A=1)$

**Practical Implication:**
- Must choose which fairness criterion to prioritize
- Trade-offs are inevitable
- Context and stakeholder input essential

---

### **7. Fairness Metrics Summary Table**

| **Metric** | **Formula** | **Focus** | **Use Case** | **Threshold** |
|------------|-------------|-----------|--------------|---------------|
| **Demographic Parity** | $P(\\hat{Y}=1\\|A=1) - P(\\hat{Y}=1\\|A=0)$ | Selection rate | Hiring, lending | $\\|\\text{DPD}\\| < 0.1$ |
| **Equal Opportunity** | $\\text{TPR}_{A=1} - \\text{TPR}_{A=0}$ | True positive rate | Medical diagnosis | $\\|\\text{EOD}\\| < 0.1$ |
| **Equalized Odds** | $\\max(\\|\\Delta \\text{TPR}\\|, \\|\\Delta \\text{FPR}\\|)$ | TPR & FPR | Criminal justice | $< 0.1$ |
| **Calibration** | $\\max_s \\|P(Y=1\\|\\hat{Y}=s, A)\\|$ | Probability accuracy | Risk assessment | $< 0.05$ |
| **Predictive Parity** | $\\text{PPV}_{A=1} - \\text{PPV}_{A=0}$ | Precision | Credit scoring | $\\|\\Delta \\text{PPV}\\| < 0.1$ |

---

### **8. Debiasing Techniques**

#### **Pre-Processing (Data Transformation)**

**Reweighting:**

$$
w_i = \\frac{P(A = a_i)}{P(A = a_i, Y = y_i)} \\cdot P(Y = y_i)
$$

Reweight samples to equalize joint distribution of $(A, Y)$.

**Sampling:**
- **Upsampling:** Duplicate minority group samples
- **Downsampling:** Remove majority group samples
- **SMOTE:** Synthetic Minority Over-sampling

**Disparate Impact Remover:**
- Modify features to decorrelate with protected attribute
- Preserve predictive power while reducing discrimination

---

#### **In-Processing (Constrained Optimization)**

**Fairness Regularization:**

$$
\\min_\\theta \\mathcal{L}(\\theta) + \\lambda \\cdot \\text{Fairness Penalty}(\\theta)
$$

Example penalties:
- Demographic Parity: $\\lambda |P(\\hat{Y}=1|A=1) - P(\\hat{Y}=1|A=0)|$
- Equal Opportunity: $\\lambda |\\text{TPR}_{A=1} - \\text{TPR}_{A=0}|$

**Adversarial Debiasing:**

$$
\\min_\\theta \\mathcal{L}_{pred}(\\theta) - \\lambda \\mathcal{L}_{adv}(\\theta)
$$

Where:
- $\\mathcal{L}_{pred}$: Prediction loss
- $\\mathcal{L}_{adv}$: Adversary's ability to predict $A$ from $\\hat{Y}$

**Adversarial Network:**
- Predictor: Maximize prediction accuracy
- Adversary: Predict protected attribute from predictions
- Goal: Predictions are accurate but uninformative about $A$

---

#### **Post-Processing (Threshold Adjustment)**

**Group-Specific Thresholds:**

Find thresholds $\\tau_0, \\tau_1$ satisfying fairness constraint:

$$
\\text{For Equal Opportunity: } P(\\hat{Y}=1|Y=1, A=0) = P(\\hat{Y}=1|Y=1, A=1)
$$

Implementation:
- Compute $\\tau_0, \\tau_1$ such that TPR is equal across groups
- Apply: $\\hat{Y}_i = \\mathbb{1}[p_i > \\tau_{A_i}]$

**Calibration via Isotonic Regression:**
- Fit isotonic regression per group
- Map raw scores to calibrated probabilities

---

### **Toy Example: Demographic Parity Calculation**

**Scenario:** Hiring model predicts 100 candidates (50 Group A, 50 Group B)

**Predictions:**
- Group A: 30 hired (60% selection rate)
- Group B: 20 hired (40% selection rate)

**Demographic Parity Difference:**

$$
\\text{DPD} = 0.40 - 0.60 = -0.20
$$

**Interpretation:**
- $|\\text{DPD}| = 0.20$: Significant bias (exceeds 0.1 threshold)
- Group B is **20% less likely** to be hired
- **Action required:** Apply debiasing technique

### 📝 Synthetic Biased Dataset Generation

**Purpose:** Create realistic dataset with built-in bias to demonstrate fairness issues.

**Dataset Characteristics:**
- **Binary classification:** Loan approval (0 = denied, 1 = approved)
- **Protected attribute:** Group membership (A: 0 = privileged, 1 = unprivileged)
- **Features:** Credit score, income, age, employment years, debt ratio
- **Built-in bias:** Unprivileged group has lower approval rate for same qualifications

**Bias Injection:**
- Base approval probability depends on legitimate features (credit score, income)
- Add systematic bias: -15% approval probability for unprivileged group
- Result: Demographic Parity violation (~20% DPD)

In [ ]:
"""
Generate Synthetic Biased Lending Dataset
==========================================

Purpose: Create realistic loan approval data with systematic bias.

Features:
- credit_score: 300-850 (FICO score)
- income: $20K-$150K (annual)
- age: 21-70 years
- employment_years: 0-40 years
- debt_ratio: 0-0.8 (debt/income)

Protected Attribute:
- group: 0 (privileged), 1 (unprivileged)

Target:
- approved: 0 (denied), 1 (approved)
"""

def generate_biased_lending_data(n_samples=5000, bias_strength=0.15, random_state=42):
    """
    Generate synthetic lending dataset with built-in bias.
    
    Args:
        n_samples: Number of samples
        bias_strength: Magnitude of bias (-0.15 = 15% lower approval for unprivileged)
        random_state: Random seed
    
    Returns:
        df: DataFrame with features, protected attribute, and biased labels
    """
    np.random.seed(random_state)
    
    # Generate protected attribute (50-50 split)
    group = np.random.binomial(1, 0.5, n_samples)
    
    # Generate legitimate features (correlated with approval)
    credit_score = np.random.normal(650, 80, n_samples)
    credit_score = np.clip(credit_score, 300, 850)
    
    income = np.random.lognormal(10.8, 0.5, n_samples)  # Mean ~$60K
    income = np.clip(income, 20000, 150000)
    
    age = np.random.normal(40, 12, n_samples)
    age = np.clip(age, 21, 70)
    
    employment_years = np.random.exponential(8, n_samples)
    employment_years = np.clip(employment_years, 0, 40)
    
    debt_ratio = np.random.beta(2, 5, n_samples) * 0.8
    
    # Compute base approval probability (legitimate factors)
    base_prob = (
        0.3 * (credit_score - 300) / 550 +  # Credit score contribution
        0.25 * (income - 20000) / 130000 +   # Income contribution
        0.15 * (employment_years / 40) +     # Employment contribution
        0.15 * (1 - debt_ratio) +            # Low debt is good
        0.15 * ((age - 21) / 49)             # Age contribution
    )
    base_prob = np.clip(base_prob, 0, 1)
    
    # Inject systematic bias: unprivileged group gets lower approval probability
    biased_prob = base_prob.copy()
    biased_prob[group == 1] -= bias_strength  # Reduce approval for unprivileged
    biased_prob = np.clip(biased_prob, 0, 1)
    
    # Generate biased labels
    approved = np.random.binomial(1, biased_prob)
    
    # Create DataFrame
    df = pd.DataFrame({
        'credit_score': credit_score,
        'income': income,
        'age': age,
        'employment_years': employment_years,
        'debt_ratio': debt_ratio,
        'group': group,
        'approved': approved
    })
    
    return df


# Generate dataset
print("Generating biased lending dataset...")
df = generate_biased_lending_data(n_samples=5000, bias_strength=0.15)

print(f"\n✅ Dataset created: {df.shape[0]} samples, {df.shape[1]-1} features")
print(f"\nProtected attribute distribution:")
print(df['group'].value_counts())
print(f"  Privileged (group=0): {(df['group'] == 0).sum()}")
print(f"  Unprivileged (group=1): {(df['group'] == 1).sum()}")

print(f"\nApproval rates (showing bias):")
approval_by_group = df.groupby('group')['approved'].agg(['mean', 'count'])
print(approval_by_group)
print(f"\n  Privileged approval rate: {approval_by_group.loc[0, 'mean']:.1%}")
print(f"  Unprivileged approval rate: {approval_by_group.loc[1, 'mean']:.1%}")
print(f"  Demographic Parity Difference: {approval_by_group.loc[1, 'mean'] - approval_by_group.loc[0, 'mean']:.3f}")

# Display sample data
print(f"\nSample data:")
print(df.head(10))

# Visualize bias
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Chart 1: Approval rates by group
ax1 = axes[0]
approval_rates = df.groupby('group')['approved'].mean()
colors = ['#44a47c', '#e74c3c']
bars = ax1.bar(['Privileged\n(Group 0)', 'Unprivileged\n(Group 1)'], 
               approval_rates.values, color=colors, edgecolor='black', linewidth=1.5, alpha=0.8)
ax1.set_ylabel('Approval Rate', fontsize=12, weight='bold')
ax1.set_title('Approval Rates by Protected Group\n(Systematic Bias Present)', fontsize=13, weight='bold', pad=15)
ax1.set_ylim(0, 1)
ax1.axhline(0.5, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax1.grid(axis='y', alpha=0.3)

for bar, rate in zip(bars, approval_rates.values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{rate:.1%}', ha='center', va='bottom', fontsize=11, weight='bold')

# Chart 2: Credit score distributions
ax2 = axes[1]
ax2.hist(df[df['group'] == 0]['credit_score'], bins=30, alpha=0.6, label='Privileged', color='#44a47c')
ax2.hist(df[df['group'] == 1]['credit_score'], bins=30, alpha=0.6, label='Unprivileged', color='#e74c3c')
ax2.set_xlabel('Credit Score', fontsize=12, weight='bold')
ax2.set_ylabel('Count', fontsize=12, weight='bold')
ax2.set_title('Credit Score Distribution by Group\n(Similar Qualifications)', fontsize=13, weight='bold', pad=15)
ax2.legend(fontsize=11)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('biased_dataset_visualization.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n📊 Key Observations:")
print("  - Unprivileged group has ~15-20% lower approval rate despite similar credit scores")
print("  - This demonstrates systematic bias in the decision-making process")
print("  - Fairness intervention is needed to achieve equitable outcomes")

### 📊 Fairness Metrics from Scratch

**Purpose:** Implement bias detection metrics to quantify unfairness.

**Key Concepts:**
- **Demographic Parity Difference (DPD)**: $P(\hat{Y}=1|G=1) - P(\hat{Y}=1|G=0)$  
  Measures difference in positive prediction rates between groups  
  **Target:** -0.1 to +0.1 (within 10%)

- **Equal Opportunity Difference (EOD)**: $TPR_1 - TPR_0$  
  Measures difference in true positive rates (among qualified individuals)  
  **Target:** -0.1 to +0.1

- **Equalized Odds**: Both TPR and FPR should be equal across groups  
  **Target:** Max(|TPR_diff|, |FPR_diff|) < 0.1

**Post-Silicon Application:**  
In device binning, ensure that devices from different wafer lots have equal chance of passing premium bins when they meet quality thresholds (prevent systematic bias against specific fabrication batches).

In [ ]:
"""
Fairness Metrics Implementation
================================

Compute bias metrics from first principles.
"""

def compute_fairness_metrics(y_true, y_pred, protected_attr):
    """
    Compute comprehensive fairness metrics.
    
    Args:
        y_true: True labels (0/1)
        y_pred: Predicted labels (0/1)
        protected_attr: Protected group (0=privileged, 1=unprivileged)
    
    Returns:
        dict: Fairness metrics
    """
    # Separate by group
    priv_mask = (protected_attr == 0)
    unpriv_mask = (protected_attr == 1)
    
    # Demographic Parity Difference
    priv_positive_rate = y_pred[priv_mask].mean()
    unpriv_positive_rate = y_pred[unpriv_mask].mean()
    dpd = unpriv_positive_rate - priv_positive_rate
    
    # Equal Opportunity Difference (TPR among qualified)
    # TPR = True Positives / (True Positives + False Negatives)
    priv_qualified = (y_true[priv_mask] == 1)
    unpriv_qualified = (y_true[unpriv_mask] == 1)
    
    priv_tpr = y_pred[priv_mask][priv_qualified].mean() if priv_qualified.sum() > 0 else 0
    unpriv_tpr = y_pred[unpriv_mask][unpriv_qualified].mean() if unpriv_qualified.sum() > 0 else 0
    eod = unpriv_tpr - priv_tpr
    
    # False Positive Rate (FPR among unqualified)
    priv_unqualified = (y_true[priv_mask] == 0)
    unpriv_unqualified = (y_true[unpriv_mask] == 0)
    
    priv_fpr = y_pred[priv_mask][priv_unqualified].mean() if priv_unqualified.sum() > 0 else 0
    unpriv_fpr = y_pred[unpriv_mask][unpriv_unqualified].mean() if unpriv_unqualified.sum() > 0 else 0
    fpr_diff = unpriv_fpr - priv_fpr
    
    # Equalized Odds (max of TPR and FPR differences)
    equalized_odds = max(abs(eod), abs(fpr_diff))
    
    # Accuracy by group
    priv_accuracy = (y_pred[priv_mask] == y_true[priv_mask]).mean()
    unpriv_accuracy = (y_pred[unpriv_mask] == y_true[unpriv_mask]).mean()
    
    return {
        'demographic_parity_diff': dpd,
        'equal_opportunity_diff': eod,
        'equalized_odds': equalized_odds,
        'fpr_diff': fpr_diff,
        'priv_positive_rate': priv_positive_rate,
        'unpriv_positive_rate': unpriv_positive_rate,
        'priv_tpr': priv_tpr,
        'unpriv_tpr': unpriv_tpr,
        'priv_fpr': priv_fpr,
        'unpriv_fpr': unpriv_fpr,
        'priv_accuracy': priv_accuracy,
        'unpriv_accuracy': unpriv_accuracy
    }


# Train baseline model (biased)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Split data
X = df[['credit_score', 'income', 'age', 'employment_years', 'debt_ratio']].values
y = df['approved'].values
group = df['group'].values

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    X, y, group, test_size=0.3, random_state=42, stratify=group
)

# Train biased model
print("Training baseline model (on biased data)...")
model_biased = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=8)
model_biased.fit(X_train, y_train)

# Predictions
y_pred_biased = model_biased.predict(X_test)

# Compute fairness metrics
print("\n" + "="*60)
print("BASELINE MODEL FAIRNESS METRICS (Biased)")
print("="*60)

metrics_biased = compute_fairness_metrics(y_test, y_pred_biased, group_test)

print(f"\n📊 Demographic Parity:")
print(f"  Privileged positive rate: {metrics_biased['priv_positive_rate']:.3f}")
print(f"  Unprivileged positive rate: {metrics_biased['unpriv_positive_rate']:.3f}")
print(f"  ➡️  DPD: {metrics_biased['demographic_parity_diff']:.3f} (target: -0.1 to 0.1)")
print(f"  ⚠️  Status: {'✅ FAIR' if abs(metrics_biased['demographic_parity_diff']) < 0.1 else '❌ BIASED'}")

print(f"\n📊 Equal Opportunity:")
print(f"  Privileged TPR: {metrics_biased['priv_tpr']:.3f}")
print(f"  Unprivileged TPR: {metrics_biased['unpriv_tpr']:.3f}")
print(f"  ➡️  EOD: {metrics_biased['equal_opportunity_diff']:.3f} (target: -0.1 to 0.1)")
print(f"  ⚠️  Status: {'✅ FAIR' if abs(metrics_biased['equal_opportunity_diff']) < 0.1 else '❌ BIASED'}")

print(f"\n📊 Equalized Odds:")
print(f"  FPR difference: {metrics_biased['fpr_diff']:.3f}")
print(f"  ➡️  Max(TPR_diff, FPR_diff): {metrics_biased['equalized_odds']:.3f}")
print(f"  ⚠️  Status: {'✅ FAIR' if metrics_biased['equalized_odds'] < 0.1 else '❌ BIASED'}")

print(f"\n📊 Accuracy by Group:")
print(f"  Privileged: {metrics_biased['priv_accuracy']:.3f}")
print(f"  Unprivileged: {metrics_biased['unpriv_accuracy']:.3f}")
print(f"  Gap: {abs(metrics_biased['priv_accuracy'] - metrics_biased['unpriv_accuracy']):.3f}")

print("\n💡 Interpretation:")
print("  - Negative DPD = Unprivileged group has lower approval rate")
print("  - Model has learned and amplified the training data bias")
print("  - Debiasing techniques are required to achieve fairness")

### 🔧 Pre-Processing Debiasing: Reweighing

**Purpose:** Assign weights to training samples to balance group representation.

**How It Works:**
1. Compute expected and observed counts for each (group, label) combination
2. Calculate weights: $W_{g,y} = \frac{P(Y=y) \cdot P(G=g)}{P(Y=y, G=g)}$
3. Train model with weighted samples

**Advantages:**
- ✅ Model-agnostic (works with any classifier)
- ✅ Simple to implement
- ✅ Preserves all training data

**Post-Silicon Application:**  
Reweight wafer test data to ensure models don't favor specific fabs or process nodes when predicting device yield. Prevents systematic undervaluation of devices from newer process nodes.

In [ ]:
"""
Pre-Processing Debiasing: Reweighing
======================================

Rebalance training data using sample weights.
"""

def compute_reweighing_weights(y, protected_attr):
    """
    Compute reweighing weights for fairness.
    
    Formula: W(g,y) = [P(Y=y) * P(G=g)] / P(Y=y, G=g)
    
    Args:
        y: Labels (0/1)
        protected_attr: Protected group (0/1)
    
    Returns:
        weights: Sample weights array
    """
    weights = np.ones(len(y))
    
    # Compute probabilities
    n = len(y)
    
    for g in [0, 1]:  # Groups
        for label in [0, 1]:  # Labels
            # P(Y=y)
            p_y = (y == label).mean()
            
            # P(G=g)
            p_g = (protected_attr == g).mean()
            
            # P(Y=y, G=g)
            mask = (y == label) & (protected_attr == g)
            p_y_g = mask.mean()
            
            # Compute weight
            if p_y_g > 0:
                weight = (p_y * p_g) / p_y_g
                weights[mask] = weight
    
    return weights


# Compute reweighing weights
print("Computing reweighing weights...")
sample_weights = compute_reweighing_weights(y_train, group_train)

print(f"\nWeight statistics:")
print(f"  Mean: {sample_weights.mean():.3f}")
print(f"  Std: {sample_weights.std():.3f}")
print(f"  Min: {sample_weights.min():.3f}")
print(f"  Max: {sample_weights.max():.3f}")

# Show weights for each (group, label) combination
for g in [0, 1]:
    for label in [0, 1]:
        mask = (y_train == label) & (group_train == g)
        if mask.sum() > 0:
            avg_weight = sample_weights[mask].mean()
            print(f"  Group {g}, Label {label}: weight={avg_weight:.3f} (n={mask.sum()})")

# Train debiased model with reweighing
print("\nTraining model with reweighing...")
model_reweighed = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=8)
model_reweighed.fit(X_train, y_train, sample_weight=sample_weights)

# Predictions
y_pred_reweighed = model_reweighed.predict(X_test)

# Compute fairness metrics
print("\n" + "="*60)
print("REWEIGHED MODEL FAIRNESS METRICS")
print("="*60)

metrics_reweighed = compute_fairness_metrics(y_test, y_pred_reweighed, group_test)

print(f"\n📊 Demographic Parity:")
print(f"  Privileged positive rate: {metrics_reweighed['priv_positive_rate']:.3f}")
print(f"  Unprivileged positive rate: {metrics_reweighed['unpriv_positive_rate']:.3f}")
print(f"  ➡️  DPD: {metrics_reweighed['demographic_parity_diff']:.3f} (target: -0.1 to 0.1)")
print(f"  ⚠️  Status: {'✅ FAIR' if abs(metrics_reweighed['demographic_parity_diff']) < 0.1 else '❌ BIASED'}")

print(f"\n📊 Equal Opportunity:")
print(f"  Privileged TPR: {metrics_reweighed['priv_tpr']:.3f}")
print(f"  Unprivileged TPR: {metrics_reweighed['unpriv_tpr']:.3f}")
print(f"  ➡️  EOD: {metrics_reweighed['equal_opportunity_diff']:.3f} (target: -0.1 to 0.1)")
print(f"  ⚠️  Status: {'✅ FAIR' if abs(metrics_reweighed['equal_opportunity_diff']) < 0.1 else '❌ BIASED'}")

print(f"\n📊 Accuracy by Group:")
print(f"  Privileged: {metrics_reweighed['priv_accuracy']:.3f}")
print(f"  Unprivileged: {metrics_reweighed['unpriv_accuracy']:.3f}")

# Compare with baseline
print("\n" + "="*60)
print("IMPROVEMENT FROM REWEIGHING")
print("="*60)
print(f"Demographic Parity Diff: {metrics_biased['demographic_parity_diff']:.3f} → {metrics_reweighed['demographic_parity_diff']:.3f}")
print(f"  Reduction: {abs(metrics_biased['demographic_parity_diff']) - abs(metrics_reweighed['demographic_parity_diff']):.3f}")
print(f"Equal Opportunity Diff: {metrics_biased['equal_opportunity_diff']:.3f} → {metrics_reweighed['equal_opportunity_diff']:.3f}")
print(f"  Reduction: {abs(metrics_biased['equal_opportunity_diff']) - abs(metrics_reweighed['equal_opportunity_diff']):.3f}")

### 🎯 Post-Processing Debiasing: Threshold Optimization

**Purpose:** Adjust decision thresholds per group to achieve fairness.

**How It Works:**
1. Train standard model (may be biased)
2. Get probability predictions
3. Find optimal thresholds for each group to satisfy fairness constraint
4. Apply group-specific thresholds at inference time

**Constraint Options:**
- **Demographic Parity**: Same positive rate across groups
- **Equal Opportunity**: Same TPR across groups
- **Equalized Odds**: Same TPR and FPR across groups

**Advantages:**
- ✅ Works with pre-trained models (no retraining needed)
- ✅ Flexible fairness constraints
- ✅ Fast to deploy

**Post-Silicon Application:**  
Optimize binning thresholds separately for different wafer fabs to ensure equal opportunity for premium bin assignment when devices meet specifications. Prevents revenue loss from overly conservative binning of specific fab outputs.

In [ ]:
"""
Post-Processing Debiasing: Threshold Optimization
==================================================

Adjust decision thresholds per group to satisfy fairness constraints.
"""

def optimize_thresholds_equal_opportunity(y_true, y_prob, protected_attr):
    """
    Find thresholds that equalize True Positive Rates across groups.
    
    Args:
        y_true: True labels
        y_prob: Predicted probabilities
        protected_attr: Protected group (0/1)
    
    Returns:
        dict: Optimal thresholds and predictions
    """
    from sklearn.metrics import roc_curve
    
    # Separate by group
    priv_mask = (protected_attr == 0)
    unpriv_mask = (protected_attr == 1)
    
    # Get ROC curves for each group
    fpr_priv, tpr_priv, thresh_priv = roc_curve(y_true[priv_mask], y_prob[priv_mask])
    fpr_unpriv, tpr_unpriv, thresh_unpriv = roc_curve(y_true[unpriv_mask], y_prob[unpriv_mask])
    
    # Find target TPR (use average of achievable TPRs)
    target_tpr = 0.75  # Aim for 75% TPR
    
    # Find thresholds closest to target TPR
    idx_priv = np.argmin(np.abs(tpr_priv - target_tpr))
    idx_unpriv = np.argmin(np.abs(tpr_unpriv - target_tpr))
    
    threshold_priv = thresh_priv[idx_priv]
    threshold_unpriv = thresh_unpriv[idx_unpriv]
    
    # Apply group-specific thresholds
    y_pred_fair = np.zeros(len(y_true), dtype=int)
    y_pred_fair[priv_mask] = (y_prob[priv_mask] >= threshold_priv).astype(int)
    y_pred_fair[unpriv_mask] = (y_prob[unpriv_mask] >= threshold_unpriv).astype(int)
    
    return {
        'threshold_priv': threshold_priv,
        'threshold_unpriv': threshold_unpriv,
        'tpr_priv': tpr_priv[idx_priv],
        'tpr_unpriv': tpr_unpriv[idx_unpriv],
        'y_pred_fair': y_pred_fair
    }


# Get probability predictions from biased model
print("Optimizing thresholds for Equal Opportunity...")
y_prob_test = model_biased.predict_proba(X_test)[:, 1]

# Optimize thresholds
result = optimize_thresholds_equal_opportunity(y_test, y_prob_test, group_test)

print(f"\n✅ Optimal Thresholds Found:")
print(f"  Privileged group: {result['threshold_priv']:.3f}")
print(f"  Unprivileged group: {result['threshold_unpriv']:.3f}")
print(f"  Target TPR achieved: ~{result['tpr_priv']:.3f} (both groups)")

# Compute fairness metrics with optimized thresholds
print("\n" + "="*60)
print("THRESHOLD-OPTIMIZED MODEL FAIRNESS METRICS")
print("="*60)

metrics_threshold = compute_fairness_metrics(y_test, result['y_pred_fair'], group_test)

print(f"\n📊 Demographic Parity:")
print(f"  DPD: {metrics_threshold['demographic_parity_diff']:.3f}")
print(f"  Status: {'✅ FAIR' if abs(metrics_threshold['demographic_parity_diff']) < 0.1 else '⚠️ PARTIAL'}")

print(f"\n📊 Equal Opportunity:")
print(f"  Privileged TPR: {metrics_threshold['priv_tpr']:.3f}")
print(f"  Unprivileged TPR: {metrics_threshold['unpriv_tpr']:.3f}")
print(f"  ➡️  EOD: {metrics_threshold['equal_opportunity_diff']:.3f}")
print(f"  ⚠️  Status: {'✅ FAIR' if abs(metrics_threshold['equal_opportunity_diff']) < 0.1 else '❌ BIASED'}")

print(f"\n📊 Accuracy by Group:")
print(f"  Privileged: {metrics_threshold['priv_accuracy']:.3f}")
print(f"  Unprivileged: {metrics_threshold['unpriv_accuracy']:.3f}")

# Comparison table
print("\n" + "="*60)
print("COMPARISON: BASELINE vs REWEIGHED vs THRESHOLD-OPTIMIZED")
print("="*60)

comparison = pd.DataFrame({
    'Metric': ['DPD', 'EOD', 'Equalized Odds', 'Priv Accuracy', 'Unpriv Accuracy'],
    'Baseline (Biased)': [
        f"{metrics_biased['demographic_parity_diff']:.3f}",
        f"{metrics_biased['equal_opportunity_diff']:.3f}",
        f"{metrics_biased['equalized_odds']:.3f}",
        f"{metrics_biased['priv_accuracy']:.3f}",
        f"{metrics_biased['unpriv_accuracy']:.3f}"
    ],
    'Reweighed': [
        f"{metrics_reweighed['demographic_parity_diff']:.3f}",
        f"{metrics_reweighed['equal_opportunity_diff']:.3f}",
        f"{metrics_reweighed['equalized_odds']:.3f}",
        f"{metrics_reweighed['priv_accuracy']:.3f}",
        f"{metrics_reweighed['unpriv_accuracy']:.3f}"
    ],
    'Threshold-Optimized': [
        f"{metrics_threshold['demographic_parity_diff']:.3f}",
        f"{metrics_threshold['equal_opportunity_diff']:.3f}",
        f"{metrics_threshold['equalized_odds']:.3f}",
        f"{metrics_threshold['priv_accuracy']:.3f}",
        f"{metrics_threshold['unpriv_accuracy']:.3f}"
    ]
})

print(comparison.to_string(index=False))

print("\n💡 Key Observations:")
print("  - Threshold optimization achieves near-perfect Equal Opportunity")
print("  - Reweighing improves Demographic Parity more effectively")
print("  - Choice of debiasing method depends on fairness constraint priority")

### 📊 Fairness Visualization & Analysis

In [ ]:
"""
Visualize Fairness Improvements
================================
"""

# Create comprehensive comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Chart 1: Demographic Parity Difference
ax1 = axes[0, 0]
models = ['Baseline\n(Biased)', 'Reweighed', 'Threshold\nOptimized']
dpd_values = [
    metrics_biased['demographic_parity_diff'],
    metrics_reweighed['demographic_parity_diff'],
    metrics_threshold['demographic_parity_diff']
]
colors = ['#e74c3c', '#f39c12', '#44a47c']
bars = ax1.bar(models, dpd_values, color=colors, edgecolor='black', linewidth=1.5, alpha=0.8)
ax1.axhline(0, color='black', linewidth=1)
ax1.axhline(0.1, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='Fair threshold')
ax1.axhline(-0.1, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax1.set_ylabel('Demographic Parity Difference', fontsize=12, weight='bold')
ax1.set_title('Demographic Parity Improvement', fontsize=13, weight='bold', pad=15)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, dpd_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + (0.01 if height > 0 else -0.02),
             f'{val:.3f}', ha='center', va='bottom' if height > 0 else 'top', 
             fontsize=11, weight='bold')

# Chart 2: Equal Opportunity Difference
ax2 = axes[0, 1]
eod_values = [
    metrics_biased['equal_opportunity_diff'],
    metrics_reweighed['equal_opportunity_diff'],
    metrics_threshold['equal_opportunity_diff']
]
bars = ax2.bar(models, eod_values, color=colors, edgecolor='black', linewidth=1.5, alpha=0.8)
ax2.axhline(0, color='black', linewidth=1)
ax2.axhline(0.1, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='Fair threshold')
ax2.axhline(-0.1, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_ylabel('Equal Opportunity Difference', fontsize=12, weight='bold')
ax2.set_title('Equal Opportunity Improvement', fontsize=13, weight='bold', pad=15)
ax2.legend()
ax2.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, eod_values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + (0.01 if height > 0 else -0.02),
             f'{val:.3f}', ha='center', va='bottom' if height > 0 else 'top',
             fontsize=11, weight='bold')

# Chart 3: Group-wise Accuracy
ax3 = axes[1, 0]
x_pos = np.arange(len(models))
width = 0.35
priv_acc = [metrics_biased['priv_accuracy'], metrics_reweighed['priv_accuracy'], metrics_threshold['priv_accuracy']]
unpriv_acc = [metrics_biased['unpriv_accuracy'], metrics_reweighed['unpriv_accuracy'], metrics_threshold['unpriv_accuracy']]

bars1 = ax3.bar(x_pos - width/2, priv_acc, width, label='Privileged', color='#44a47c', 
                edgecolor='black', linewidth=1, alpha=0.8)
bars2 = ax3.bar(x_pos + width/2, unpriv_acc, width, label='Unprivileged', color='#3498db',
                edgecolor='black', linewidth=1, alpha=0.8)

ax3.set_ylabel('Accuracy', fontsize=12, weight='bold')
ax3.set_title('Accuracy by Protected Group', fontsize=13, weight='bold', pad=15)
ax3.set_xticks(x_pos)
ax3.set_xticklabels(models)
ax3.legend()
ax3.set_ylim(0.5, 1.0)
ax3.grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{height:.3f}', ha='center', va='bottom', fontsize=10)

# Chart 4: Fairness-Accuracy Trade-off
ax4 = axes[1, 1]
avg_acc = [
    (metrics_biased['priv_accuracy'] + metrics_biased['unpriv_accuracy']) / 2,
    (metrics_reweighed['priv_accuracy'] + metrics_reweighed['unpriv_accuracy']) / 2,
    (metrics_threshold['priv_accuracy'] + metrics_threshold['unpriv_accuracy']) / 2
]
fairness_score = [
    1 - abs(metrics_biased['demographic_parity_diff']),
    1 - abs(metrics_reweighed['demographic_parity_diff']),
    1 - abs(metrics_threshold['demographic_parity_diff'])
]

ax4.scatter(fairness_score, avg_acc, s=300, c=colors, edgecolor='black', linewidth=2, alpha=0.8, zorder=3)
for i, model in enumerate(models):
    ax4.annotate(model.replace('\n', ' '), 
                 (fairness_score[i], avg_acc[i]),
                 xytext=(10, 10), textcoords='offset points',
                 fontsize=10, weight='bold',
                 bbox=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='black', alpha=0.7))

ax4.set_xlabel('Fairness Score (1 - |DPD|)', fontsize=12, weight='bold')
ax4.set_ylabel('Average Accuracy', fontsize=12, weight='bold')
ax4.set_title('Fairness-Accuracy Trade-off', fontsize=13, weight='bold', pad=15)
ax4.grid(alpha=0.3)
ax4.set_xlim(0.7, 1.05)
ax4.set_ylim(0.65, 0.85)

plt.suptitle('Comprehensive Fairness Analysis: Debiasing Techniques Comparison', 
             fontsize=16, weight='bold', y=0.995)
plt.tight_layout()
plt.savefig('fairness_comparison.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n" + "="*70)
print("FAIRNESS DEBIASING SUMMARY")
print("="*70)
print("\n✅ BEST FOR DEMOGRAPHIC PARITY: Reweighed")
print(f"   - DPD reduced from {metrics_biased['demographic_parity_diff']:.3f} to {metrics_reweighed['demographic_parity_diff']:.3f}")
print(f"   - Improvement: {abs(metrics_biased['demographic_parity_diff']) - abs(metrics_reweighed['demographic_parity_diff']):.3f}")

print("\n✅ BEST FOR EQUAL OPPORTUNITY: Threshold-Optimized")
print(f"   - EOD reduced from {metrics_biased['equal_opportunity_diff']:.3f} to {metrics_threshold['equal_opportunity_diff']:.3f}")
print(f"   - Improvement: {abs(metrics_biased['equal_opportunity_diff']) - abs(metrics_threshold['equal_opportunity_diff']):.3f}")

print("\n💡 Recommendations:")
print("  1. Use Reweighing for group-level fairness (hiring, lending)")
print("  2. Use Threshold Optimization for outcome fairness (medical, legal)")
print("  3. Monitor fairness-accuracy trade-offs continuously")
print("  4. Document fairness constraints in model cards")

## 🏭 Real-World Project Ideas

Build production-ready fairness systems for impactful applications.

---

### **Project 1: Fair Device Binning System** ⭐ Post-Silicon

**Objective:** Ensure equitable premium bin assignment across different wafer fabrication plants.

**Business Value:** $18.2M/year revenue protection by preventing systematic undervaluation of devices from specific fabs.

**Dataset:**
- Parametric test data (Vdd, Idd, frequency, temperature)
- Wafer metadata (fab_id, process_node, lot_id)
- Bin assignments (premium, standard, low-power)

**Fairness Constraint:** Equal Opportunity - devices meeting premium specs should have equal probability of premium binning regardless of fab origin.

**Implementation:**
1. Train multi-class binning classifier
2. Detect fab-level bias in premium bin assignments
3. Apply threshold optimization per fab
4. Validate fairness metrics (EOD < 0.1 across all fabs)
5. Monitor fairness drift over time

**Success Metrics:**
- Demographic Parity Diff < 0.08 across fabs
- Equal Opportunity Diff < 0.1 for premium bin
- Revenue recovery: $18.2M/year from fair binning

---

### **Project 2: Equitable ATE Resource Allocation** ⭐ Post-Silicon

**Objective:** Fair distribution of premium Automatic Test Equipment (ATE) across product lines.

**Business Value:** $24.6M/year efficiency gain by preventing systematic over-allocation to legacy products.

**Dataset:**
- Product test requirements (test_time, complexity, volume)
- ATE capabilities (speed, accuracy, cost_per_hour)
- Historical allocation patterns (product_id, ate_id, utilization)

**Fairness Constraint:** Calibrated fairness - allocation probability should match actual test needs, not legacy bias.

**Implementation:**
1. Build ATE allocation recommendation system
2. Compute fairness metrics across product lines
3. Apply reweighing to balance historical over-representation
4. Implement allocation policy with fairness constraints
5. Track allocation equity over quarters

**Success Metrics:**
- Allocation fairness score > 0.92 across products
- Utilization improvement: +18% for under-allocated products
- Cost savings: $24.6M/year from optimal allocation

---

### **Project 3: Bias-Free Root Cause Analysis** ⭐ Post-Silicon

**Objective:** Prevent systematic misattribution of test failures to specific process steps.

**Business Value:** $12.8M/year cost avoidance by accurate failure diagnosis (prevent unnecessary process changes).

**Dataset:**
- Failure signatures (test_name, failure_mode, parametric_values)
- Process history (etch_time, deposition_thickness, anneal_temp)
- Actual root causes (verified through failure analysis)

**Fairness Constraint:** Individual fairness - similar failure patterns should receive similar root cause assignments regardless of process step.

**Implementation:**
1. Train root cause classifier
2. Measure attribution bias across process steps
3. Apply adversarial debiasing (remove process_step signal)
4. Validate with failure analysis ground truth
5. Deploy with fairness monitoring dashboard

**Success Metrics:**
- Attribution accuracy > 87% (up from 72%)
- Demographic Parity Diff < 0.12 across process steps
- False root cause reduction: 64% → 18%

---

### **Project 4: Fair Supplier Performance Benchmarking** ⭐ Post-Silicon

**Objective:** Equitable evaluation of component suppliers to prevent bias against new/smaller vendors.

**Business Value:** $8.4M/year cost reduction from competitive supplier ecosystem (prevent monopoly pricing).

**Dataset:**
- Component quality metrics (defect_rate, reliability, test_yield)
- Supplier attributes (size, tenure, geography, price)
- Historical evaluation scores

**Fairness Constraint:** Equalized Odds - suppliers with similar quality should receive similar scores regardless of size/tenure.

**Implementation:**
1. Build supplier scoring model
2. Detect bias against new/small suppliers
3. Apply reweighing to balance historical data
4. Implement fair scoring policy
5. Track supplier diversity and cost competitiveness

**Success Metrics:**
- Equalized Odds metric < 0.1 (size/tenure groups)
- New supplier adoption: +42% qualified vendors
- Cost reduction: $8.4M/year from competitive pricing

---

### **Project 5: Fair Hiring and Promotion System** 🌍 General

**Objective:** Build bias-free candidate evaluation system for tech recruiting.

**Business Value:** $16M/year productivity gain from diverse talent + legal risk reduction.

**Dataset:**
- Candidate profiles (education, experience, skills, projects)
- Interview scores (technical, behavioral, culture_fit)
- Historical hiring decisions and performance data

**Fairness Constraint:** Demographic Parity + Equal Opportunity across gender/race.

**Implementation:**
1. Train candidate ranking model
2. Audit for demographic bias
3. Apply adversarial debiasing + reweighing
4. Implement fair interview panel selection
5. Monitor hiring funnel fairness metrics

**Success Metrics:**
- DPD < 0.08 across protected groups
- EOD < 0.1 for high-performer identification
- Diversity hiring: +35% underrepresented candidates

---

### **Project 6: Fair Credit Scoring System** 🌍 General

**Objective:** Equitable loan approval system that eliminates historical lending bias.

**Business Value:** $48M/year market expansion from underserved communities + regulatory compliance.

**Dataset:**
- Credit history (payment_history, credit_utilization, age_of_credit)
- Income and employment data
- Loan repayment outcomes

**Fairness Constraint:** Equal Opportunity - qualified borrowers should have equal approval rates regardless of demographics.

**Implementation:**
1. Train credit risk model
2. Measure bias across demographic groups
3. Apply threshold optimization per group
4. Validate with regulatory fairness standards (ECOA compliance)
5. Deploy with real-time bias monitoring

**Success Metrics:**
- Equal Opportunity Diff < 0.1
- Default rate maintained < 3.2%
- Market expansion: +$48M/year revenue from fair lending

---

### **Project 7: Fair Medical Diagnosis Assistant** 🌍 General

**Objective:** Ensure equitable diagnostic accuracy across patient demographics.

**Business Value:** $120M/year healthcare cost reduction from early diagnosis + malpractice risk mitigation.

**Dataset:**
- Patient records (symptoms, lab_results, imaging, demographics)
- Diagnostic labels (disease, severity)
- Treatment outcomes

**Fairness Constraint:** Equalized Odds - same sensitivity/specificity across age/gender/race.

**Implementation:**
1. Train diagnostic classifier (disease detection)
2. Audit for demographic disparities in sensitivity
3. Apply group-specific threshold optimization
4. Validate on diverse test cohorts
5. Deploy with clinical fairness dashboard

**Success Metrics:**
- Equalized Odds < 0.08 across demographics
- Sensitivity improvement: +12% for under-diagnosed groups
- Mortality reduction: 18% from equitable early diagnosis

---

### **Project 8: Fair Criminal Risk Assessment** 🌍 General

**Objective:** Bias-free recidivism prediction for bail/sentencing decisions.

**Business Value:** $280M/year social value (reduced wrongful incarceration + recidivism prevention).

**Dataset:**
- Criminal history (offense_type, severity, frequency)
- Demographics (age, race, socioeconomic_status)
- Recidivism outcomes (rearrest, conviction)

**Fairness Constraint:** Calibration + Equal Opportunity - risk scores should be calibrated and predictive accuracy equal across groups.

**Implementation:**
1. Train recidivism prediction model
2. Measure calibration bias across demographics
3. Apply post-processing calibration + threshold optimization
4. Validate with external fairness auditors
5. Deploy with judicial oversight and transparency

**Success Metrics:**
- Calibration error < 0.05 across race
- Equal Opportunity Diff < 0.1
- Wrongful detention reduction: 34%

---

### 🎯 Project Selection Guide

| **Constraint Priority** | **Recommended Technique** | **Best Projects** |
|------------------------|---------------------------|-------------------|
| Demographic Parity | Reweighing | Hiring (P5), Supplier (P4) |
| Equal Opportunity | Threshold Optimization | Credit (P6), Medical (P7), Binning (P1) |
| Equalized Odds | Adversarial Debiasing | Criminal Justice (P8), Root Cause (P3) |
| Calibration | Post-processing | Medical (P7), ATE Allocation (P2) |

## 🎓 Key Takeaways

### ✅ When to Use Fairness & Bias Mitigation

**Use fairness-aware ML when:**
- Decisions affect protected groups (hiring, lending, healthcare, criminal justice)
- Historical data contains systematic bias
- Regulatory compliance required (ECOA, GDPR Article 22, NYC AI Law)
- Stakeholder trust and social responsibility are priorities
- Post-silicon: Preventing systematic bias in device evaluation across fabs/batches

**Skip when:**
- No protected attributes involved (weather forecasting, game AI)
- Individual predictions don't affect people's opportunities
- Exploratory analysis without deployment

---

### 🎯 Choosing the Right Fairness Metric

| **Metric** | **Use Case** | **Post-Silicon Example** |
|-----------|-------------|--------------------------|
| **Demographic Parity** | Group representation | Ensure wafer lots get equal test coverage |
| **Equal Opportunity** | Qualified outcome equity | Devices meeting specs get premium bins equally |
| **Equalized Odds** | Full predictive parity | Root cause attribution accurate across process steps |
| **Calibration** | Risk score reliability | Reliability predictions calibrated across product lines |
| **Individual Fairness** | Similar treatment | Similar devices receive similar binning regardless of fab |

---

### 🔧 Debiasing Technique Selection

**Pre-Processing (Reweighing, Resampling):**
- ✅ Model-agnostic (works with any classifier)
- ✅ Easy to implement and explain
- ❌ May discard valuable data (resampling)
- **Best for:** Demographic Parity, when fairness is top priority

**In-Processing (Adversarial Debiasing, Fairness Regularization):**
- ✅ Integrated into training (single model)
- ✅ Can optimize fairness-accuracy trade-off
- ❌ Requires custom training loops
- **Best for:** Complex fairness constraints, neural networks

**Post-Processing (Threshold Optimization, Calibration):**
- ✅ Works with pre-trained models (no retraining)
- ✅ Fast to deploy and tune
- ❌ May sacrifice overall accuracy
- **Best for:** Equal Opportunity, Equalized Odds, when model is frozen

---

### ⚠️ Limitations & Challenges

1. **Fairness-Accuracy Trade-off:**  
   Stricter fairness constraints often reduce overall accuracy. Document and justify trade-offs.

2. **Impossibility Theorem:**  
   Cannot simultaneously satisfy Demographic Parity, Equal Opportunity, and Calibration (except in trivial cases). Choose priority metric based on application.

3. **Protected Attribute Unavailability:**  
   If protected attributes aren't collected (privacy/legal reasons), use proxy detection or fairness-through-unawareness (risky).

4. **Fairness Shift Over Time:**  
   Data distribution changes can reintroduce bias. Continuous monitoring essential.

5. **Definition Disagreement:**  
   Stakeholders may disagree on fairness definition. Transparent communication required.

6. **Post-Silicon Challenge:**  
   Fab-specific process variations may legitimately correlate with yield. Distinguish bias from causal differences.

---

### 🚀 Best Practices

**1. Fairness Auditing Pipeline:**
```python
# Standard fairness audit workflow
1. Identify protected attributes (gender, race, fab_id, etc.)
2. Compute baseline bias metrics (DPD, EOD, calibration)
3. Apply debiasing technique(s)
4. Validate fairness improvement on holdout set
5. Document fairness-accuracy trade-offs
6. Deploy with continuous monitoring
```

**2. Model Cards for Fairness:**
- Document fairness metrics in model cards
- Report performance disaggregated by protected groups
- Explain debiasing techniques applied
- State limitations and known biases

**3. Human-in-the-Loop:**
- Fairness-aware ML supports decisions, doesn't replace human judgment
- High-stakes applications (bail, medical) require human oversight
- Provide explanations alongside fairness-adjusted predictions

**4. Continuous Monitoring:**
- Track fairness metrics in production (weekly/monthly)
- Set alerts for fairness degradation (DPD > 0.15, EOD > 0.12)
- Retrain and re-debias when drift detected

---

### 📊 Post-Silicon Specific Insights

**1. Fab-Level Fairness:**
- Treat fabrication plants as protected groups
- Ensure devices from different fabs with similar parametrics receive equal treatment
- Revenue impact: $18M/year from fair binning

**2. Process Node Equity:**
- Newer process nodes may have different yield characteristics (legitimate)
- Distinguish between legitimate causal differences and historical bias
- Apply fairness constraints only to comparable quality levels

**3. Supplier Fairness:**
- Prevent systematic bias against smaller/newer component suppliers
- Cost savings: $8.4M/year from competitive supplier ecosystem

**4. Test Coverage Fairness:**
- Ensure premium ATE resources allocated fairly across products
- Efficiency gain: $24.6M/year from equitable resource allocation

---

### 🔗 Next Steps in MLOps

After mastering fairness & bias mitigation:

1. **Model Governance (177):** Implement policies and audit trails for fair ML systems
2. **Privacy-Preserving ML (178):** Combine fairness with differential privacy
3. **Ethical AI Frameworks (179):** Broader ethical considerations beyond fairness
4. **Regulatory Compliance (180):** EU AI Act, NYC AI Law, ECOA compliance

**Recommended Reading:**
- "Fairness and Machine Learning" by Barocas, Hardt, Narayanan
- Google's "ML Fairness Guidelines"
- Microsoft's "Responsible AI Standard"
- IEEE 7000-2021 (Systems Design for Ethical Concerns)

---

### 💡 Final Thought

**Fairness is not optional** - it's a fundamental requirement for trustworthy AI systems. Whether preventing discriminatory lending, ensuring equitable healthcare, or achieving fair device binning in semiconductors, fairness-aware ML protects both people and business value.

**The cost of unfairness:**
- Legal liability (class-action lawsuits, regulatory fines)
- Reputational damage (PR crises, lost customers)
- Social harm (perpetuating inequality)
- Lost revenue (market exclusion, monopolistic supplier pricing)

**The value of fairness:**
- Market expansion ($48M/year from fair credit)
- Talent diversity ($16M/year productivity gain)
- Regulatory compliance (avoid fines, maintain licensing)
- Social responsibility (measurable positive impact)

---

Let's build fair AI systems that work for everyone! 🌍

## 🔑 Key Takeaways

**When to Use Fairness/Bias Analysis:**
- High-stakes decisions affecting people (hiring, lending, healthcare, criminal justice)
- Regulatory requirements (GDPR, ECOA, Fair Housing Act)
- Protected attributes in data (race, gender, age, disability)
- Public-facing AI systems (brand reputation risk from unfair outcomes)

**Limitations:**
- Fairness metrics can conflict (demographic parity vs equalized odds - can't optimize both)
- Debiasing may reduce overall accuracy (fairness-accuracy trade-off)
- Requires domain expertise (what constitutes "fair" varies by context)
- Can't fix biased data collection (garbage in, garbage out)
- Post-hoc fixes don't address root causes (need fair data collection processes)

**Alternatives:**
- **Fair data collection** (balanced sampling, auditing data sources)
- **Causal modeling** (understand causal relationships, not just correlations)
- **Human-in-the-loop** (manual review for edge cases)
- **Reject option classification** (defer ambiguous cases to humans)

**Best Practices:**
- Define fairness metric early (align with stakeholders before building)
- Measure multiple fairness metrics (demographic parity, equalized odds, calibration)
- Use adversarial debiasing or reweighting (not just post-processing)
- Monitor fairness in production (drift can reintroduce bias over time)
- Document bias mitigation efforts (model cards, fairness reports)
- Involve diverse stakeholders (ethicists, domain experts, affected communities)

**Next Steps:**
- 177: Privacy-Preserving ML (privacy and fairness often aligned)
- 178: AI Safety & Alignment (value alignment, robustness to adversaries)
- 127: Model Governance (audit trails for fairness compliance)